In [1]:
# ==========================================================
#  ESM Benchmark

import torch, esm, pandas as pd, numpy as np, time, os, warnings
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}\n")

# ----------------------------------------------------------
dataset = pd.read_excel("KRc.xlsx", usecols=["sequence", "label"])
sequences = dataset["sequence"].astype(str).tolist()
labels = dataset["label"].to_numpy()

# ----------------------------------------------------------
def get_embeddings(model_name, save_name, batch_size=8):
    print(f"\n[{model_name}] Loading model...")
    model, alphabet = getattr(esm.pretrained, model_name)()
    batch_converter = alphabet.get_batch_converter()
    model.eval().to(device)
    num_layers = model.num_layers
    print(f"Model loaded ✓  ({sum(p.numel() for p in model.parameters())/1e6:.1f} M params)")

    if os.path.exists(save_name):
        print(f"🔹 Found cached file: {save_name}")
        return pd.read_csv(save_name)

    all_embs = []
    print(f"Generating embeddings ({len(sequences)} sequences)...")
    for i in tqdm(range(0, len(sequences), batch_size)):
        batch = [(f"seq_{j}", seq) for j, seq in enumerate(sequences[i:i+batch_size])]
        _, _, batch_tokens = batch_converter(batch)
        batch_tokens = batch_tokens.to(device)
        with torch.no_grad():
            results = model(batch_tokens, repr_layers=[num_layers], return_contacts=False)
        reps = results["representations"][num_layers]
        seq_reps = [
            reps[k, 1:(batch_tokens[k] != alphabet.padding_idx).sum()-1].mean(0).cpu().numpy()
            for k in range(len(batch))
        ]
        all_embs.extend(seq_reps)
        torch.cuda.empty_cache()

    df = pd.DataFrame(np.array(all_embs))
    df.to_csv(save_name, index=False)
    print(f"✅ Saved → {save_name}\n")
    return df

# ----------------------------------------------------------
model_list = {
    "esm1b_t33_650M_UR50S": "ESM1b_650M.csv",
    "esm2_t6_8M_UR50D":     "ESM2_8M.csv",
    "esm2_t33_650M_UR50D":  "ESM2_650M.csv",
    "esm2_t36_3B_UR50D":    "ESM2_3B.csv",
}

# ----------------------------------------------------------
def evaluate_classifiers_cv(X, y, n_splits=5):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    classifiers = {
        "SVM (RBF)": SVC(kernel="rbf", probability=True, random_state=42),
        "Logistic Regression": LogisticRegression(max_iter=2000, random_state=42),
        "Random Forest": RandomForestClassifier(
            n_estimators=300, class_weight="balanced", random_state=42
        ),
        "XGBoost": XGBClassifier(
            use_label_encoder=False, eval_metric="logloss", random_state=42
        ),
    }

    results = []
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    for name, clf in classifiers.items():
        acc_scores, auc_scores = [], []
        pA, rA, fA, pB, rB, fB = [], [], [], [], [], []
        print(f"\n🔹 {name} ({n_splits}-fold CV)")

        for fold, (train_idx, test_idx) in enumerate(cv.split(X_scaled, y), 1):
            X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            y_prob = (
                clf.predict_proba(X_test)[:, 1]
                if hasattr(clf, "predict_proba")
                else y_pred
            )

            acc = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_prob)
            report = classification_report(y_test, y_pred, output_dict=True)

            acc_scores.append(acc)
            auc_scores.append(auc)
            pA.append(report["1"]["precision"])
            rA.append(report["1"]["recall"])
            fA.append(report["1"]["f1-score"])
            pB.append(report["0"]["precision"])
            rB.append(report["0"]["recall"])
            fB.append(report["0"]["f1-score"])

            print(f"  Fold {fold}: acc={acc:.4f}, auc={auc:.4f}")

        results.append({
            "Classifier": name,
            "Accuracy_mean": np.mean(acc_scores),
            "Accuracy_std": np.std(acc_scores),
            "AUC_mean": np.mean(auc_scores),
            "AUC_std": np.std(auc_scores),
            "Precision (Type A)": np.mean(pA),
            "Recall (Type A)": np.mean(rA),
            "F1-Score (Type A)": np.mean(fA),
            "Precision (Type B)": np.mean(pB),
            "Recall (Type B)": np.mean(rB),
            "F1-Score (Type B)": np.mean(fB),
        })

    return pd.DataFrame(results)

# ----------------------------------------------------------
excel_path = "ESM_Benchmark_5fold.xlsx"
summary = []

with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    for model_name, file_name in model_list.items():
        print(f"\n=== Evaluating {model_name} ===")
        t0 = time.time()

        df = get_embeddings(model_name, file_name)
        X = df.to_numpy()

        res = evaluate_classifiers_cv(X, labels, n_splits=5)
        res["Model"] = model_name
        summary.append(res)

        sheet = model_name.replace("esm", "").replace("_UR50D","").replace("_UR50S","")[:31]
        res.to_excel(writer, sheet_name=sheet, index=False)

        print(f"⏱  {model_name} done in {(time.time()-t0)/60:.1f} min\n")

    summary_df = pd.concat(summary)
    summary_df = summary_df[
        [
            "Model", "Classifier",
            "Accuracy_mean", "Accuracy_std",
            "AUC_mean", "AUC_std",
            "Precision (Type A)", "Recall (Type A)", "F1-Score (Type A)",
            "Precision (Type B)", "Recall (Type B)", "F1-Score (Type B)"
        ]
    ]
    summary_df.to_excel(writer, sheet_name="Summary_All", index=False)

print(f"\n✅ All results saved to {excel_path}\n")
print("🏁 Final Summary")
print(summary_df)


Using device: cuda


=== Evaluating esm1b_t33_650M_UR50S ===

[esm1b_t33_650M_UR50S] Loading model...
Model loaded ✓  (652.4 M params)
🔹 Found cached file: ESM1b_650M.csv

🔹 SVM (RBF) (5-fold CV)
  Fold 1: acc=0.9126, auc=0.9496
  Fold 2: acc=0.9612, auc=0.9636
  Fold 3: acc=0.9903, auc=0.9977
  Fold 4: acc=0.9612, auc=0.9807
  Fold 5: acc=0.9118, auc=0.9838

🔹 Logistic Regression (5-fold CV)
  Fold 1: acc=0.9223, auc=0.9523
  Fold 2: acc=0.9612, auc=0.9643
  Fold 3: acc=0.9709, auc=0.9775
  Fold 4: acc=0.9515, auc=0.9773
  Fold 5: acc=0.9706, auc=0.9673

🔹 Random Forest (5-fold CV)
  Fold 1: acc=0.9126, auc=0.9626
  Fold 2: acc=0.9612, auc=0.9715
  Fold 3: acc=0.9515, auc=0.9934
  Fold 4: acc=0.9126, auc=0.9744
  Fold 5: acc=0.9020, auc=0.9732

🔹 XGBoost (5-fold CV)
  Fold 1: acc=0.9029, auc=0.9605
  Fold 2: acc=0.9515, auc=0.9647
  Fold 3: acc=0.9515, auc=0.9922
  Fold 4: acc=0.9515, auc=0.9753
  Fold 5: acc=0.9118, auc=0.9622
⏱  esm1b_t33_650M_UR50S done in 0.2 min


=== Evaluating 

In [ ]:
#non_strep_sequences

In [25]:
import pandas as pd

# ----------------------------------------------------------
file_path = "non_strep_sequences.xlsx"
df_A = pd.read_excel(file_path, sheet_name="A")
df_B = pd.read_excel(file_path, sheet_name="B")

print("Sheet A columns:", list(df_A.columns))
print("Sheet B columns:", list(df_B.columns))

# ----------------------------------------------------------
def detect_sequence_column(df):
    for col in df.columns:
        if df[col].astype(str).str.match(r'^[A-Z\-]{30,}$', na=False).sum() > len(df) * 0.5:
            return col
    return df.columns[-1]  # fallback

seq_col_A = detect_sequence_column(df_A)
seq_col_B = detect_sequence_column(df_B)

print(f"Detected sequence column in A: {seq_col_A}")
print(f"Detected sequence column in B: {seq_col_B}")

# ----------------------------------------------------------
def remove_gaps(seq):
    if pd.isna(seq):
        return None
    return seq.replace("-", "").replace(" ", "").strip()

df_A["sequence_clean"] = df_A[seq_col_A].apply(remove_gaps)
df_B["sequence_clean"] = df_B[seq_col_B].apply(remove_gaps)

# ----------------------------------------------------------
df_A["label"] = 1
df_B["label"] = 0

# ----------------------------------------------------------
dataset = pd.concat([df_A, df_B], ignore_index=True)
dataset = dataset.dropna(subset=["sequence_clean"])  # drop blanks

print(f"✅ Loaded {len(dataset)} total sequences after cleaning")
print(dataset[["label", "sequence_clean"]].head())

dataset.to_csv("KRc_nonStreptomyces_cleaned.csv", index=False)
print("Saved → KRc_nonStreptomyces_cleaned.csv")


Sheet A columns: ['aetokthonostatin', 'aesB', 'M6', 'A1KR', 'Aetokthonos hydrillicola Thurmond2011', 'Cyanobacteriota', 'GVYLIFGGLGRVGLLLALDLAQRVKAKLVLVSRSELPPRYKIRNLIDIEKAGGEVMTIAADVTDLDQMRSLR-RIQ--GQIHGVFHAAAITS-NRFLKEVGIEESTVQFAPKVFGLYLLEEVFGQ--PDFFVLLSSNASILGGLGFCAYSAANHFLDYFAADRNHQAVPWISTNWDRWLLE-HE---EAYT--TSMDAFAMTPQQSLEAMRRIITQ--EQVVVTPGNLQDRLKIWIQ----QSKQNN']
Sheet B columns: ['BGC0000102.1 ', 'mycAI', 'M1', 'B1KR', 'Micromonospora griseorubida', 'Actinomycetes', 'GTVLVTGGTGALGARVARWLADHGAEHLVLVSRRGAQATGAAELVAGLERTGVRVTVAACDVADRDGAGRPAGGAPAGRPDGGPAHSRGSGC---------------GVPAERLAATHLHDLTAHR-LDAFVLFSSIVGSWGNAGQAAYAAANAALDALAEQRRSAGLPATSIAWGLWAGG-MAD---GEQTFTRRGVRAMDPDDGIAALRQALDS--TCVTVADVDWPSMVRTHANPAAARLFEEI']
Detected sequence column in A: GVYLIFGGLGRVGLLLALDLAQRVKAKLVLVSRSELPPRYKIRNLIDIEKAGGEVMTIAADVTDLDQMRSLR-RIQ--GQIHGVFHAAAITS-NRFLKEVGIEESTVQFAPKVFGLYLLEEVFGQ--PDFFVLLSSNASILGGLGFCAYSAANHFLDYFAADRNHQAVPWISTNWDRWLLE-HE---EAYT--TSMDAFAMTPQQSLEAMRRIITQ--EQVVVTPGNLQDRLKIWIQ----

In [3]:
import torch, esm, pandas as pd, numpy as np, time
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score,
    recall_score, f1_score
)
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}\n")

# ----------------------------------------------------------
file_path = "KRc_nonStreptomyces_cleaned.csv"
dataset = pd.read_csv(file_path)

seq_col = "sequence_clean" if "sequence_clean" in dataset.columns else dataset.columns[-1]
sequences = dataset[seq_col].astype(str).tolist()
labels = dataset["label"].to_numpy()

print(f"Total sequences: {len(sequences)} | A-type: {sum(labels)} | B-type: {len(labels)-sum(labels)}\n")

# ----------------------------------------------------------
def get_embeddings(model_name="esm2_t36_3B_UR50D", batch_size=4):
    print(f"\n[{model_name}] Loading model...")
    model, alphabet = getattr(esm.pretrained, model_name)()
    batch_converter = alphabet.get_batch_converter()
    model.eval().to(device)
    num_layers = model.num_layers
    print(f"Model loaded ✓  ({sum(p.numel() for p in model.parameters())/1e6:.1f} M params)\n")

    all_embs = []
    for i in tqdm(range(0, len(sequences), batch_size)):
        batch = [(f"seq_{j}", seq) for j, seq in enumerate(sequences[i:i+batch_size])]
        _, _, batch_tokens = batch_converter(batch)
        batch_tokens = batch_tokens.to(device)
        with torch.no_grad():
            results = model(batch_tokens, repr_layers=[num_layers], return_contacts=False)
        reps = results["representations"][num_layers]
        seq_reps = [
            reps[k, 1:(batch_tokens[k] != alphabet.padding_idx).sum()-1].mean(0).cpu().numpy()
            for k in range(len(batch))
        ]
        all_embs.extend(seq_reps)
        torch.cuda.empty_cache()

    emb_df = pd.DataFrame(np.array(all_embs))
    emb_df.to_csv("ESM2_3B_nonStreptomyces.csv", index=False)
    print("Embeddings saved → ESM2_3B_nonStreptomyces.csv\n")
    return emb_df

X = get_embeddings()

# ----------------------------------------------------------
def cross_validate_classifiers(X, y, n_splits=5):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    classifiers = {
        "SVM (RBF)": SVC(kernel="rbf", probability=True, random_state=42),
        "Logistic Regression": LogisticRegression(max_iter=2000, random_state=42),
        "Random Forest": RandomForestClassifier(n_estimators=300, class_weight="balanced", random_state=42),
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
    }

    results = []

    for name, clf in classifiers.items():
        acc_scores, auc_scores, prec_scores, rec_scores, f1_scores = [], [], [], [], []
        print(f"\n🔹 Running {n_splits}-fold CV for {name} ...")

        for fold, (train_idx, test_idx) in enumerate(skf.split(X_scaled, y), 1):
            X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            y_prob = clf.predict_proba(X_test)[:, 1] if hasattr(clf, "predict_proba") else y_pred

            acc = accuracy_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_prob)
            prec = precision_score(y_test, y_pred)
            rec = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)

            acc_scores.append(acc)
            auc_scores.append(auc)
            prec_scores.append(prec)
            rec_scores.append(rec)
            f1_scores.append(f1)

            print(f"  Fold {fold}: Acc={acc:.4f}, AUC={auc:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}")

        results.append((
            name,
            np.mean(acc_scores), np.std(acc_scores),
            np.mean(auc_scores), np.std(auc_scores),
            np.mean(prec_scores), np.std(prec_scores),
            np.mean(rec_scores), np.std(rec_scores),
            np.mean(f1_scores), np.std(f1_scores)
        ))

    results_df = pd.DataFrame(results, columns=[
        "Classifier",
        "Mean_Accuracy", "Std_Accuracy",
        "Mean_AUC", "Std_AUC",
        "Mean_Precision", "Std_Precision",
        "Mean_Recall", "Std_Recall",
        "Mean_F1", "Std_F1"
    ])
    return results_df

results_df = cross_validate_classifiers(X.to_numpy(), labels)
results_df.to_csv("ESM2_3B_nonStreptomyces_5fold_fullmetrics.csv", index=False)

print("\n🏁 Final 5-Fold CV Results")
print(results_df)


Using device: cuda

Total sequences: 201 | A-type: 114 | B-type: 87


[esm2_t36_3B_UR50D] Loading model...
Model loaded ✓  (2839.0 M params)



100%|███████████████████████████████████████████| 51/51 [00:08<00:00,  5.69it/s]


Embeddings saved → ESM2_3B_nonStreptomyces.csv


🔹 Running 5-fold CV for SVM (RBF) ...
  Fold 1: Acc=0.9024, AUC=0.9469, Prec=0.9130, Rec=0.9130, F1=0.9130
  Fold 2: Acc=1.0000, AUC=1.0000, Prec=1.0000, Rec=1.0000, F1=1.0000
  Fold 3: Acc=0.9500, AUC=0.9974, Prec=1.0000, Rec=0.9130, F1=0.9545
  Fold 4: Acc=0.9000, AUC=0.9770, Prec=0.9524, Rec=0.8696, F1=0.9091
  Fold 5: Acc=0.9000, AUC=0.9848, Prec=0.9091, Rec=0.9091, F1=0.9091

🔹 Running 5-fold CV for Logistic Regression ...
  Fold 1: Acc=0.9268, AUC=0.9396, Prec=0.9167, Rec=0.9565, F1=0.9362
  Fold 2: Acc=0.9500, AUC=0.9949, Prec=0.9565, Rec=0.9565, F1=0.9565
  Fold 3: Acc=0.9500, AUC=0.9565, Prec=0.9200, Rec=1.0000, F1=0.9583
  Fold 4: Acc=0.9000, AUC=0.9668, Prec=0.9524, Rec=0.8696, F1=0.9091
  Fold 5: Acc=0.9000, AUC=0.9848, Prec=0.9091, Rec=0.9091, F1=0.9091

🔹 Running 5-fold CV for Random Forest ...
  Fold 1: Acc=0.9024, AUC=0.9191, Prec=0.9130, Rec=0.9130, F1=0.9130
  Fold 2: Acc=1.0000, AUC=1.0000, Prec=1.0000, Rec=1.0000, F1=